In [1]:
include('init.lua')

In [2]:
torch.setnumthreads(1)
net = lstm.LSTM({num_layers=2, input_dim=2, hidden_dim=3})

In [3]:
inputs = torch.randn(5,4,2)
net:forget()
result = net:forward(inputs)
print(result:size())

 5
 4
 2
 3
[torch.LongStorage of size 4]



In [4]:
grads = torch.randn(5,4,2,3)
in_grads = net:backward(inputs, grads)
print(in_grads)

(1,.,.) = 
 -0.0778 -0.0319
 -0.0978 -0.1188
 -0.0953  0.0287
 -0.1729  0.0514

(2,.,.) = 
  0.1702 -0.0831
  0.1492 -0.1974
  0.1533 -0.1580
 -0.0322 -0.0553

(3,.,.) = 
 -0.0391  0.0544
  0.0629 -0.0524
 -0.3844  0.3833
 -0.0528  0.0347

(4,.,.) = 
  0.2700 -0.0418
  0.1353 -0.1419
 -0.0529  0.1051
  0.0248 -0.0862

(5,.,.) = 
  0.0910  0.0046
  0.0936 -0.2985
  0.1209 -0.1718
  0.0730 -0.0933
[torch.DoubleTensor of size 5x4x2]



In [2]:
self = {num_layers = 2, input_dim=2, hidden_dim=3}
local input = nn.Identity()()
local c_p = nn.Identity()()
local h_p = nn.Identity()()
local inputs = {input, c_p, h_p} -- {x_t, c_{t-1}, h_{t-1}}
local h, c = {}, {}


for l = 1, self.num_layers do
    local c_l_p = nn.SelectTable(l)(c_p)
    local h_l_p = nn.SelectTable(l)(h_p)
    local i2h
    if l == 1 then
        i2h = nn.Linear(self.input_dim, 4*self.hidden_dim)(input) -- W_x * x_t + b_x
    else
        i2h = nn.Linear(self.hidden_dim, 4*self.hidden_dim)(h[l-1]) -- W_x * h_{t-1} + b_x
    end
    local h2h = nn.Linear(self.hidden_dim, 4*self.hidden_dim)(h_l_p) -- W_h * h_{t-1} + b_h  
    -- preactivations for i_t, f_t, o_t, c_in_t (update)
    local preacts = nn.CAddTable()({i2h, h2h}) -- i2h + h2h

    -- direction of Narrow = 2 (row vector input)
    -- nonlinear:
    --     input, forget, and output gates get Sigmoid
    --     state update gets Tanh
    local all_gates = nn.Sigmoid()(nn.Narrow(2, 1, 3*self.hidden_dim)(preacts)) 
    local update = nn.Tanh()(nn.Narrow(2, 3*self.hidden_dim + 1, self.hidden_dim)(preacts))
    -- split gates into their variables
    local i_gate = nn.Narrow(2, 1, self.hidden_dim)(all_gates)
    local f_gate = nn.Narrow(2, self.hidden_dim + 1, self.hidden_dim)(all_gates)
    local o_gate = nn.Narrow(2, 2 * self.hidden_dim + 1, self.hidden_dim)(all_gates)
    -- new state, c = f_t .* c_p + i_t .* c_in_t
    c[l] = nn.CAddTable()({
            nn.CMulTable()({f_gate, c_l_p}),
            nn.CMulTable()({i_gate, update})
        })
    -- new hidden, h = o_t .* Tanh(c)
    h[l] = nn.CMulTable()({
            o_gate,
            nn.Tanh()(c[l])
        })
end
local outputs = {nn.Identity()(c), nn.Identity()(h)} -- output new state c, and new hidden h

cell = nn.gModule(inputs, outputs)


In [16]:
result = cell:forward({
        torch.randn(5,2), 
        {torch.randn(5,3),torch.randn(5,3)}, 
        {torch.randn(5,3),torch.randn(5,3)}
    }
)

In [17]:
result

{
  1 : 
    {
      1 : DoubleTensor - size: 5x3
      2 : DoubleTensor - size: 5x3
    }
  2 : 
    {
      1 : DoubleTensor - size: 5x3
      2 : DoubleTensor - size: 5x3
    }
}


In [18]:
cell:backward({torch.Tensor{5,2}, {torch.randn(5,3), torch.randn(5,3)}, {torch.randn(5,3), torch.randn(5,3)}},
    {{torch.randn(5,3),torch.randn(5,3)}, {torch.randn(5,3),torch.randn(5,3)}}
    )

{
  1 : DoubleTensor - size: 5x2
  2 : 
    {
      1 : DoubleTensor - size: 5x3
      2 : DoubleTensor - size: 5x3
    }
  3 : 
    {
      1 : DoubleTensor - size: 5x3
      2 : DoubleTensor - size: 5x3
    }
}


In [38]:
a = torch.randn(5,2,4)

In [41]:
a[{{}, {}, 1}]

 1.1834  0.9655
-0.5355  0.8005
 4.3073  0.5039
 0.2041  0.0605
 0.2387 -0.3394
[torch.DoubleTensor of size 5x2]



In [42]:
a:size()


 5
 2
 4
[torch.LongStorage of size 3]



In [43]:
#a:size()

3	


In [44]:
b= torch.randn(2,4)

In [45]:
b:reshape(1,2,4)

(1,.,.) = 
  0.2341 -0.6311 -1.8598 -1.4613
  0.3113  2.0795  0.7851 -1.4057
[torch.DoubleTensor of size 1x2x4]



In [46]:
b

 0.2341 -0.6311 -1.8598 -1.4613
 0.3113  2.0795  0.7851 -1.4057
[torch.DoubleTensor of size 2x4]

